In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FullyConnectedNetwork(nn.Module):
    def __init__(self):
        super(FullyConnectedNetwork, self).__init__()
        
        # First fully connected layer
        self.fc1 = nn.Linear(25 * 25, 512)
        self.dropout1 = nn.Dropout(0.5)

        # Second fully connected layer
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)

        # Third fully connected layer
        self.fc3 = nn.Linear(256, 128)

        # Output layer
        self.fc4 = nn.Linear(128, 15)

    def forward(self, x):
        # Flatten the image input
        x = x.view(x.size(0), -1)

        # Pass through the first layer, apply ReLU and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        # Second layer
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        # Third layer
        x = F.relu(self.fc3(x))

        # Output layer with softmax activation
        x = self.fc4(x)
        x = F.log_softmax(x, dim=1)
        return x # x is log probability

# Create the neural network
model = FullyConnectedNetwork()

# Print the model structure
print(model)


FullyConnectedNetwork(
  (fc1): Linear(in_features=625, out_features=512, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=15, bias=True)
)


Define a custom DataLoader for our dataset format

In [2]:
class StormShadow(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, :, :]
        label = self.labels[idx, :]
        return sample, label

Early stopping logic

In [9]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                
early_stopping = EarlyStopping(patience=50, min_delta=0.01)

# Load the data

In [3]:
import xarray as xr 
vil_ds = xr.open_zarr('../hrrr17x.zarr/')
vil_dr = vil_ds.VIL
# Get all the file names in bystormlabels folder
from tqdm import tqdm
import numpy as np 
import os

labels = np.zeros((5625, 15))

for date_str in tqdm(vil_dr.attrs['date']):
    # Replace : with _ in date_str
    filename_to_open = date_str.replace(':', '_')
    # check if the file exists
    if not os.path.isfile('bystormlabels/' + filename_to_open + '.npz'):
        continue
    labell = np.load('bystormlabels/' + filename_to_open + '.npz')['arr_0']
    frequency_vector = np.array([(labell==i).sum() for i in range(15)])
    if frequency_vector.sum() == 0:
        continue
    else:
        print(vil_dr.attrs['date'].index(date_str))
    labels[vil_dr.attrs['date'].index(date_str)] = frequency_vector / frequency_vector.sum()
vil_input = vil_dr.values

100%|██████████| 5625/5625 [00:00<00:00, 520402.33it/s]

3414


In [4]:
# Convert to tensor
vil_tensor = torch.from_numpy(vil_input)
labels_tensor = torch.from_numpy(labels) # in normal form, NOT log probability

In [5]:
def normalize_vil_input_by_max(vil_tensor):
    vil_tensor = vil_tensor - vil_tensor.min()
    vil_tensor = vil_tensor / vil_tensor.max()
    return vil_tensor

def normalize_vil_input_by_mean_and_std(vil_tensor):
    vil_tensor = vil_tensor - vil_tensor.mean()
    vil_tensor = vil_tensor / vil_tensor.std()
    return vil_tensor

vil_tensor = normalize_vil_input_by_mean_and_std(vil_tensor)

In [6]:
# Split the data into training, validation and test sets
train_split = 0.8
val_split = 0.1
test_split = 0.1

from torch.utils.data import random_split

# Calculate lengths of splits
train_length = int(train_split * len(vil_tensor))
val_length = int(val_split * len(vil_tensor))
test_length = len(vil_tensor) - train_length - val_length

# Split the data
train_data, val_data, test_data = random_split(vil_tensor, [train_length, val_length, test_length])

# Print the size of the splits
print('Train data length: ', len(train_data))
print('Val data length: ', len(val_data))
print('Test data length: ', len(test_data))

Train data length:  4500
Val data length:  562
Test data length:  563


In [7]:
# Training setup
# Create dataset
dataset = StormShadow(vil_tensor, labels_tensor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Define the model
model = FullyConnectedNetwork()

# Define the loss
criterion = nn.KLDivLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Number of epochs to train the model
n_epochs = 10

In [8]:
# Tensorboard setup
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/stormshadow')

In [15]:
# Train and Evaluation Loops

def train_one_epoch(model, train_loader, loss_function, optimizer, device):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        # Forward pass
        output = model(data)
        loss = loss_function(output, target)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate(model, data_loader, loss_function, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_function(output, target)
            total_loss += loss.item()

    return total_loss / len(data_loader)

def train_model(model, train_loader, valid_loader, loss_function, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, loss_function, optimizer, device)
        valid_loss = evaluate(model, valid_loader, loss_function, device)
        
        # Record the training loss
        writer.add_scalar('Loss/train', train_loss / len(train_loader), epoch)
        writer.add_scalar('Loss/valid', valid_loss / len(valid_loader), epoch)
        writer.add_scalar('Accuracy/train', 1 - train_loss / len(train_loader), epoch)
        writer.add_scalar('Accuracy/valid', 1 - valid_loss / len(valid_loader), epoch)
        
        # Early stopping
        early_stopping(valid_loss)

        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

# Assuming a CUDA-capable device is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
num_epochs = 10  # Example number of epochs
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

# Evaluate on the test set
test_loss = evaluate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}')

/Users/thinhhoang/miniforge3/envs/traffic/lib/python3.9/site-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 1/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 2/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 3/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 4/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 5/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 6/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 7/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 8/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 9/10, Training Loss: 0.0000, Validation Loss: 0.0000
Epoch 10/10, Training Loss: 0.0000, Validation Loss: 0.0000
Test Loss: 0.0000
